# Amazon Textract


## 3. Anaylzing Results for Asynchronous operations

In [ ]:
import sys

In [ ]:
!{sys.executable} -m pip install pdf2image

In [ ]:
#Detects text in a document stored in an S3 bucket. Display polygon box around text and angled text 
import boto3
import io
from io import BytesIO
import sys

import json
import psutil
import time
import copy
import math
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import numpy as np
import textract.util as tu
from io import BytesIO
from pdf2image import convert_from_path, convert_from_bytes
import textract.util as tu

s3_client = boto3.client('s3')

%matplotlib inline

In [ ]:
%store -r

In [ ]:
test_document

In [ ]:
jobids

### Input a document number among test_documents :

In [ ]:
document_id = 1

In [ ]:
doc_block = tu.get_multipages_block_result(jobids[document_id-1])

### 3-1) Convert PDF to Image

In [ ]:
pdf_byte_string = s3_client.get_object(Bucket=bucket, Key=test_document[document_id-1])['Body'].read()
image = convert_from_bytes(pdf_byte_string)

### 3-2) Checking the numbers of  image and doc_block for multipage documents

In [ ]:
num_img = len(image)
num_dblock = len(doc_block)
if not num_img==num_dblock:
    assert "The numbers of documents and blocks is different."
else:
    print("Numbers of documents: {}, and of blocks: {}".format(num_img, num_dblock))

### Input a page number in the selected document:

In [ ]:
page_num = 1

### 3-3) Post-processing for Analyzing Document
the location and geometry of key-value pairs, tables, cells, and selection elements.
 - Bounding box: height, left (X coordinate), top (Y coordinate), and width as a ratio of the overall document page height, left, top, and width, repectively
 - Polygon: points in the polygon array to display a finer-grain bounding box around a Block object. 

Multiply the X coordinate by the document page width, and multiply the Y coordinate by the document page height

In [ ]:
result_image, result_blocks = tu.get_multipages_analyze_document(image[page_num-1], doc_block[page_num-1])

### 3-4) Display items location on a document page

In [ ]:
fig_x, fig_y = 20, 15
plt.figure(figsize = (fig_x,fig_y))
plt.imshow(np.array(result_image))

### 3-5) The results of analyzing Text in a document page

In [ ]:
page = tu.get_page(result_blocks)

### 4. Detecting Entitiy for Amazon Comprehend

In [ ]:
## Input text size exceeds limit. Max length of request text allowed is 5000 bytes
tu.detect_entities_for_comprehend(page[:5000])


### 5. Extracting Key-Value Pairs from a Form Document

In [ ]:
key_map, value_map, block_map = tu.get_kv_map(result_blocks)

# Get Key Value relationship
kvs = tu.get_kv_relationship(key_map, value_map, block_map)
print("\n\n== FOUND KEY : VALUE pairs ===\n")
tu.print_kvs(kvs)

# Start searching a key value
while input('\n Do you want to search a value for a key? (enter "n" for exit) ') != 'n':
    search_key = input('\n Enter a search key:')
    print('The value is:', tu.search_value(kvs, search_key))